In [112]:
import requests
import json

from OTXv2 import OTXv2
from stix2 import Indicator
otx = OTXv2("7ef390234ffa11dd340269f1ae184cf9fe3191bd2ba16b23148fe33c4ebe0c91")

from stix2 import Indicator, parse

def search_domain(domain, section):
    endpoint = f"https://otx.alienvault.com/api/v1/indicators/domain/{domain}/{section}"
    try:
        response = requests.get(endpoint)
        response.raise_for_status()  # raise an exception for non-200 status codes
        data = response.json()
        return data
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return None

# alternative implementation
def json_to_stix2(domain, section):
    data = search_domain(domain, section)

    # determines the type of indicator for STIX 2.1
    if section == 'general':
        data['type'] = 'identifier'
        # No values for required properties for Indicator: (pattern, pattern_type)
    elif section == 'geo':
        data['type'] = 'location'
    elif section == 'malware':
        data['type'] = 'malware'
        # No values for required properties for Malware: (is_family)
    elif section == 'url_list':
        data['type'] = 'url'
        # No values for required properties for URL: (value)
    elif section == 'passive_dns':
        data['type'] = 'domain-name'
    elif section == 'whois':
        data['type'] = 'identity'
        # No values for required properties for Identity: (name)
    elif section == 'http_scans':
        data['type'] = 'observed-data'
        # No values for required properties for Observed Data: (first_observed, last_observed, number_observed)
        # You'd need to make your own script to keep track of this data

    try:
        # Check if required properties are present in data dictionary
        if data.get('type') == 'indicator' and not all(key in data for key in ['pattern', 'pattern_type']):
            raise ValueError("Required properties (pattern, pattern_type) missing in data")
        if data.get('type') == 'url' and not 'value' in data:
            raise ValueError("Required property (value) missing in data")
        
        if data.get('type') == 'malware' and not 'is_family' in data:
            raise ValueError("Required property (is_family) missing in data")
        
        if data.get('type') == 'identity' and not 'name' in data:
            raise ValueError("Required property (name) missing in data")
        
        if data.get('type') == 'observed-data' and not all(key in data for key in ['first_observed', 'last_observed', 'number_observed']):
            raise ValueError("Required properties (first_observed, last_observed, number_observed) missing in data")
    except ValueError as e:
        print(f"Error: {e}")
        return None

    json_data = json.dumps(data)

    # Create STIX 2.1 indicator from JSON string using parse()
    indicator = parse(json_data, allow_custom=True, version="2.1")
    indicator.serialize()  # serialize the STIX 2.1 object to JSON
    return indicator


# Get the STIX 2.1 object for the domain and section
domain = 'ualr.edu'
section = 'general'

stix_object = json_to_stix2(domain, section)
print(stix_object)
result = search_domain(domain, section)
print(result)

AttributeError: 'dict' object has no attribute 'serialize'

In [ ]:
# Mapping of section names to STIX object types
SECTION_TYPE_MAP = {
    'general': 'indicator',
    'geo': 'location',
    'malware': 'malware',
    'url_list': 'url',
    'passive_dns': 'domain-name',
    'whois': 'identity',
    'http_scans': 'network-traffic',
}


def json_to_stix(domain, section):
    # Load JSON data from API response
    json_data = search_domain(domain, section)
    
    # Determine the corresponding STIX object type
    stix_type = SECTION_TYPE_MAP.get(section, 'indicator')
    
    # Create STIX 2.1 object from JSON data
    if stix_type == 'indicator':
        stix_object = Indicator.from_dict(json_data)
    else:
        stix_object = parse(json.dumps(json_data), stix_type)
    
    return stix_object